#SYSC 5906:
## **Room Guessing - V1.0**

---
Script to use custom room detection network to "guess" which room a requested object could most likely be found in. 

###Step 1: Access Drive
Mount the drive with the provided .zip file of code located in it

In [1]:
#Enter the gdrive
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


##Step 2: Setup
Import relevenat libraries, load original dataset into colabs (without the images) and get the folders inside

In [12]:
import tensorflow as tf
import pandas as pd
import pickle
import sklearn as sk
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds
import keras.api._v2.keras as keras
from tensorflow.keras.utils import to_categorical
from keras.layers import Activation, Dense, Dropout, Input, Flatten, Embedding, Masking, LSTM, Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.models import Sequential, Model
from keras.optimizers import adam_v2
from keras.engine.input_layer import InputLayer
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.backend import flatten
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

MODEL_DIRECTORY = '/gdrive/My Drive/Colab Notebooks/SYSC 5906/models/weights/V2 Improved versions/room_classifier_3/' #Room detector/guessor 
PICKLE_DIRECTORY = '/gdrive/My Drive/Colab Notebooks/SYSC 5906/datasets/mit_indoors/processed/data_labelsOnly/'

In [13]:
#Import a subset of the MIT dataset
pickledData = open(PICKLE_DIRECTORY+"listOfAllObj_v3.pkl","rb")
dataSet = pickle.load(pickledData)
pickledData.close()

#Import list of unqiue objects
pickledObjs = open(PICKLE_DIRECTORY+"uniqueObjs_v3.pkl","rb")
uniqueObjs = pickle.load(pickledObjs)
uniqueObjs = dataSet.columns[0:-1]
pickledObjs.close()

#Load the model
model = tf.keras.models.load_model(MODEL_DIRECTORY)

#Summarize model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 2204)              0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 64)                141120    
                                                                 
 Hidden_Layer_2 (Dense)      (None, 32)                2080      
                                                                 
 Hidden_Layer_4 (Dense)      (None, 10)                330       
                                                                 
 Output_Layer (Dense)        (None, 7)                 77        
                                                                 
Total params: 143,607
Trainable params: 143,607
Non-trainable params: 0
_________________________________________________________________


**Get the target object from user:**

In [17]:
#Get a desired object from the user
targetObj = input('Enter the desired object: ')
#print(f'You entered - {targetObj}')

Enter the desired object: bed


##Step 3: Guess the room
Performs room guessing

In [15]:
def roomGuesser(targetObj, uniqueObjs, model):
    #Get list of unique object names from dataset (column titles EXCEPT room name)
    objectList = pd.DataFrame(uniqueObjs)
    #Check that user inputed desired object (target) is valid
    if targetObj not in objectList.values:
        error = ["Invalid input object"]
        return error, 0

    #Init empty vector that is the same size of unique objects list
    oneHotSearchInput = np.zeros(len(objectList))

    #Find the number that corresponds to the object label/string
    location = objectList[objectList.eq(targetObj).any(1)].index.values[0]

    #Potential rooms, used to map from number to room label/string
    desiredRooms = ['bathroom','bedroom','dining_room','corridor','livingroom','kitchen','office']

    #Set input to true at index of target object
    oneHotSearchInput[location] = 1

    #Convert vector to tensor
    oneHotTensor = tf.constant(oneHotSearchInput,shape=(1,len(uniqueObjs)))

    #Predict romm based on tensor of objects
    target_pred = model.predict(oneHotTensor)

    #Sort results into a tuple (links model probabilities with room labels/strings)
    results = []
    for room in range(len(desiredRooms)):
        results.append((target_pred[0][room],desiredRooms[room]))
    sorted_results = sorted(results,key=lambda x : x[0])

    #Find the top n most likely results
    n=3
    top_n = sorted_results[-n:]

    #Print the top n results
    print("Target: " + targetObj + " will most likely be found in:")
    for i in range(1,n+1,1):
        print(" "+top_n[-i][1] + " (" + str(top_n[-i][0]) + ")")

    return top_n, n

In [16]:
#Search for singular objects in the current input and return top results
result, n = roomGuesser(targetObj, uniqueObjs, model)

#Check if result is valid
if n == 0:
    print(result[0])
    #Get a new desired object from the user
    targetObj = input('Enter a valid desired object: ')

Target: cup will most likely be found in:
 dining_room (0.6734647)
 office (0.14224076)
 livingroom (0.11703573)
